In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from scipy.stats import mode
from torch.utils.data import DataLoader
from torchvision import models
from tqdm import tqdm

In [2]:
cinic_directory = 'C:/Users/Dell/.cache/kagglehub/datasets/mengcius/cinic10/versions/1'
cinic_mean = [0.47889522, 0.47227842, 0.43047404]
cinic_std = [0.24205776, 0.23828046, 0.25874835]

transform_augment = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform_augment_advanced = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)
])

transform_crop = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform_flip = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform_rotation = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform_colorJitter = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

transform_cutOut = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0)
])

cinic_train = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform),
    batch_size=128, shuffle=True)
cinic_valid = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid', transform=transform),
    batch_size=128, shuffle=False)

cinic_train_augment = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_augment),
    batch_size=128, shuffle=True)
cinic_train_crop = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_crop),
    batch_size=128, shuffle=True)
cinic_train_flip = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_flip),
    batch_size=128, shuffle=True)
cinic_train_rotation = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_rotation),
    batch_size=128, shuffle=True)
cinic_train_colorJitter = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_colorJitter),
    batch_size=128, shuffle=True)
cinic_train_augment_advanced = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_augment_advanced),
    batch_size=128, shuffle=True)
cinic_train_cutOut = DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train', transform=transform_cutOut),
    batch_size=128, shuffle=True)

In [3]:
cinic_mean_tensor = torch.tensor(cinic_mean).view(3, 1, 1)
cinic_std_tensor = torch.tensor(cinic_std).view(3, 1, 1)


def imshow(img):
    print(img.size())
    img = img * cinic_std_tensor + cinic_mean_tensor  # Odwracamy normalizację
    npimg = img.numpy()
    plt.figure(figsize=(0.5, 0.5))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


data_iter = iter(cinic_train)
images, labels = next(data_iter)

imshow(images[0])
print("Etykieta:", labels[0].item())

torch.Size([3, 32, 32])


Etykieta: 8


In [4]:
def train(model, device, cinic_train, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_loader_tqdm = tqdm(cinic_train, desc=f"Epoka {epoch + 1}/{num_epochs}")

        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_loader_tqdm.set_postfix(loss=loss.item())

        avg_loss = running_loss / len(cinic_train)
        accuracy = 100 * correct / total
        print(f"Epoka [{epoch + 1}/{num_epochs}], Strata: {avg_loss:.4f}, Dokładność: {accuracy:.2f}%")

    print("Trenowanie zakończone!")

In [5]:
def test(model, device, cinic_valid):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in cinic_valid:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Dokładność na zbiorze walidacyjnym: {100 * correct / total:.2f}%")

### Simple Fully Connected NN

In [24]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3 * 32 * 32, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.layers(x)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN().to(device)
model_crop = SimpleNN().to(device)
model_flip = SimpleNN().to(device)
model_rotate = SimpleNN().to(device)
model_colorJitter = SimpleNN().to(device)
model_augmentation = SimpleNN().to(device)
train(model, device, cinic_train)
train(model_crop, device, cinic_train_crop)
train(model_flip, device, cinic_train_flip)
train(model_rotate, device, cinic_train_rotation)
train(model_colorJitter, device, cinic_train_colorJitter)
train(model_augmentation, device, cinic_train_augment)


Epoka 1/10: 100%|██████████| 704/704 [00:57<00:00, 12.30it/s, loss=1.73]


Epoka [1/10], Strata: 1.9092, Dokładność: 30.61%


Epoka 2/10: 100%|██████████| 704/704 [00:56<00:00, 12.48it/s, loss=1.57]


Epoka [2/10], Strata: 1.7786, Dokładność: 35.73%


Epoka 3/10: 100%|██████████| 704/704 [01:05<00:00, 10.70it/s, loss=1.86]


...

Epoka [8/10], Strata: 1.7824, Dokładność: 35.13%


Epoka 9/10: 100%|██████████| 704/704 [01:16<00:00,  9.18it/s, loss=1.96]


Epoka [9/10], Strata: 1.7748, Dokładność: 35.59%


Epoka 10/10: 100%|██████████| 704/704 [01:15<00:00,  9.35it/s, loss=1.39]

Epoka [10/10], Strata: 1.7659, Dokładność: 35.70%
Trenowanie zakończone!


In [27]:
test(model, device, cinic_valid)
test(model_crop, device, cinic_valid)
test(model_flip, device, cinic_valid)
test(model_rotate, device, cinic_valid)
test(model_colorJitter, device, cinic_valid)
test(model_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 43.99%
Dokładność na zbiorze walidacyjnym: 39.18%
Dokładność na zbiorze walidacyjnym: 43.90%
Dokładność na zbiorze walidacyjnym: 42.42%
Dokładność na zbiorze walidacyjnym: 43.79%
Dokładność na zbiorze walidacyjnym: 35.98%


In [28]:
torch.save(model, "base.pth")
torch.save(model_crop, "base_crop.pth")
torch.save(model_flip, "base_flip.pth")
torch.save(model_rotate, "base_rotate.pth")
torch.save(model_colorJitter, "base_colorJitter.pth")
torch.save(model_augmentation, "base_augmented.pth")

### Simple CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

### VGG Style CNN

In [ ]:
class VGGStyleCNN(nn.Module):
    def __init__(self):
        super(VGGStyleCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [27]:
# test(model_vgg, device, cinic_valid)
test(model_vgg_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 78.61%


### ResNet

In [ ]:
resnet18_augmentation = torch.load("resnet18_aug.pth", weights_only=False)
resnet18 = torch.load("resnet18_no_aug.pth", weights_only=False)

In [37]:
resnet18_pretrained_augmentation = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet18_pretrained_augmentation.fc = nn.Linear(512, 10)
resnet18_pretrained_augmentation.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# resnet18 = models.resnet18(num_classes=10).to(device)
# resnet18_augmentation = models.resnet18(num_classes=10).to(device)
train(resnet18_pretrained_augmentation, device, cinic_train_augment, num_epochs=1)
# train(resnet18_augmentation, device, cinic_train_augment, num_epochs = 5)

Epoka 1/1: 100%|██████████| 704/704 [21:32<00:00,  1.84s/it, loss=1.2]  

📊 Epoka [1/1], Strata: 0.9330, Dokładność: 66.78%
✅ Trenowanie zakończone!


In [43]:
test(resnet18_pretrained_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 68.86%


In [44]:
# torch.save(resnet18, "resnet18_no_aug.pth")
torch.save(resnet18_pretrained_augmentation, "resnet18_pre_aug.pth")

### Larger VGG

In [ ]:
class VGGImprovedCNN(nn.Module):
    def __init__(self):
        super(VGGImprovedCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 1 * 1, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = VGGImprovedCNNCNN().to(device)
larger_vgg_model_augmentation = VGGImprovedCNN().to(device)
# train(model, device, cinic_train)
train(larger_vgg_model_augmentation, device, cinic_train_augment, num_epochs=30)

Epoka 1/30: 100%|██████████| 704/704 [03:46<00:00,  3.11it/s, loss=2.09]


Epoka [1/30], Strata: 1.6461, Dokładność: 38.69%


Epoka 2/30: 100%|██████████| 704/704 [03:47<00:00,  3.10it/s, loss=1.4]  


Epoka [2/30], Strata: 1.3229, Dokładność: 52.21%


Epoka 3/30: 100%|██████████| 704/704 [03:47<00:00,  3.09it/s, loss=1.25] 


...

Epoka [28/30], Strata: 0.4591, Dokładność: 83.60%


Epoka 29/30: 100%|██████████| 704/704 [03:45<00:00,  3.12it/s, loss=0.492]


Epoka [29/30], Strata: 0.4490, Dokładność: 83.98%


Epoka 30/30: 100%|██████████| 704/704 [03:44<00:00,  3.13it/s, loss=0.296]

Epoka [30/30], Strata: 0.4350, Dokładność: 84.60%
Trenowanie zakończone!


In [12]:
test(larger_vgg_model_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 78.68%


In [11]:
# torch.save(resnet18, "resnet18_no_aug.pth")
torch.save(larger_vgg_model_augmentation, "vgg_improved_aug2.pth")

### Nice comparison of augmenation

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_vgg = VGGStyleCNN().to(device)
# model_vgg_crop = VGGStyleCNN().to(device)
# model_vgg_rotation = VGGStyleCNN().to(device)
# model_vgg_flip = VGGStyleCNN().to(device)
# model_vgg_colorJitter = VGGStyleCNN().to(device)
# model_vgg_augmentation = VGGStyleCNN().to(device)
# model_vgg_augmentation_advanced = VGGStyleCNN().to(device)
# model_vgg_cutOut = VGGStyleCNN().to(device)
# model_vgg = torch.load("vgg.pth", weights_only=False)
# model_vgg_crop = torch.load("vgg_crop.pth", weights_only=False)
# model_vgg_rotation = torch.load("vgg_rotation.pth", weights_only=False)
# model_vgg_flip = torch.load("vgg_flip.pth", weights_only=False)
# model_vgg_colorJitter = torch.load("vgg_colorJitter.pth", weights_only=False)
# model_vgg_augmentation = torch.load("vgg_augmentation.pth", weights_only=False)
# train(model_vgg, device, cinic_train, num_epochs = 15)
# train(model_vgg_crop, device, cinic_train_crop, num_epochs = 15)
# train(model_vgg_flip, device, cinic_train_flip, num_epochs = 11)
# train(model_vgg_rotation, device, cinic_train_rotation, num_epochs = 15)
# train(model_vgg_colorJitter, device, cinic_train_colorJitter, num_epochs = 15)
# train(model_vgg_augmentation, device, cinic_train_augment, num_epochs = 15)
train(model_vgg_augmentation_advanced, device, cinic_train_augment_advanced, num_epochs=5)
# train(model_vgg_cutOut, device, cinic_train_cutOut, num_epochs = 15)

Epoka 1/5: 100%|██████████| 704/704 [14:42<00:00,  1.25s/it, loss=1.02] 


Epoka [1/5], Strata: 0.5856, Dokładność: 79.49%


Epoka 2/5: 100%|██████████| 704/704 [13:40<00:00,  1.16s/it, loss=0.558]


Epoka [2/5], Strata: 0.5759, Dokładność: 79.78%


Epoka 3/5: 100%|██████████| 704/704 [14:08<00:00,  1.20s/it, loss=0.389]


Epoka [3/5], Strata: 0.5716, Dokładność: 79.85%


Epoka 4/5: 100%|██████████| 704/704 [13:43<00:00,  1.17s/it, loss=0.848]


Epoka [4/5], Strata: 0.5626, Dokładność: 80.21%


Epoka 5/5: 100%|██████████| 704/704 [13:20<00:00,  1.14s/it, loss=1.03] 

Epoka [5/5], Strata: 0.5505, Dokładność: 80.67%
Trenowanie zakończone!


In [14]:
test(model_vgg, device, cinic_valid)
test(model_vgg_crop, device, cinic_valid)
test(model_vgg_flip, device, cinic_valid)
test(model_vgg_rotation, device, cinic_valid)
test(model_vgg_colorJitter, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 67.66%
Dokładność na zbiorze walidacyjnym: 74.09%
Dokładność na zbiorze walidacyjnym: 72.49%
Dokładność na zbiorze walidacyjnym: 71.00%
Dokładność na zbiorze walidacyjnym: 69.56%


In [42]:
# test(model_vgg_cutOut, device, cinic_valid)
test(model_vgg_augmentation_advanced, device, cinic_valid)
# test(model_vgg_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 78.98%


In [43]:
# torch.save(model_vgg, "vgg.pth")
# torch.save(model_vgg_crop, "vgg_crop.pth")
# torch.save(model_vgg_flip, "vgg_flip.pth")
# torch.save(model_vgg_rotation, "vgg_rotation.pth")
# torch.save(model_vgg_colorJitter, "vgg_colorJitter.pth")
# torch.save(model_vgg_augmentation, "vgg_augmentation.pth")
torch.save(model_vgg_augmentation_advanced, "vgg_augmentation_advanced_trained.pth")
# torch.save(model_vgg_cutOut, "vgg_cutOut.pth")

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_simple = CNN().to(device)
# model_simple_crop = CNN().to(device)
# model_simple_rotation = CNN().to(device)
# model_simple_flip = CNN().to(device)
# model_simple_colorJitter = CNN().to(device)
# model_simple_augmentation = CNN().to(device)
train(model_simple, device, cinic_train, num_epochs=5)
train(model_simple_crop, device, cinic_train, num_epochs=5)
train(model_simple_rotation, device, cinic_train, num_epochs=5)
train(model_simple_flip, device, cinic_train, num_epochs=5)
train(model_simple_colorJitter, device, cinic_train, num_epochs=5)
train(model_simple_augmentation, device, cinic_train, num_epochs=5)

Epoka 1/5: 100%|██████████| 704/704 [01:09<00:00, 10.14it/s, loss=1.13] 


Epoka [1/5], Strata: 0.9794, Dokładność: 65.83%


Epoka 2/5: 100%|██████████| 704/704 [01:32<00:00,  7.64it/s, loss=1.4]  


Epoka [2/5], Strata: 0.9611, Dokładność: 66.39%


Epoka 3/5: 100%|██████████| 704/704 [01:32<00:00,  7.64it/s, loss=0.536]


...

Epoka [3/5], Strata: 0.9488, Dokładność: 66.70%


Epoka 4/5: 100%|██████████| 704/704 [00:49<00:00, 14.32it/s, loss=1.02] 


Epoka [4/5], Strata: 0.9298, Dokładność: 67.53%


Epoka 5/5: 100%|██████████| 704/704 [00:49<00:00, 14.21it/s, loss=0.764]

Epoka [5/5], Strata: 0.9080, Dokładność: 68.42%
Trenowanie zakończone!


In [19]:
test(model_simple, device, cinic_valid)
test(model_simple_crop, device, cinic_valid)
test(model_simple_flip, device, cinic_valid)
test(model_simple_rotation, device, cinic_valid)
test(model_simple_colorJitter, device, cinic_valid)
test(model_simple_augmentation, device, cinic_valid)

Dokładność na zbiorze walidacyjnym: 59.75%
Dokładność na zbiorze walidacyjnym: 59.65%
Dokładność na zbiorze walidacyjnym: 60.04%
Dokładność na zbiorze walidacyjnym: 60.30%
Dokładność na zbiorze walidacyjnym: 59.95%
Dokładność na zbiorze walidacyjnym: 60.16%


In [20]:
torch.save(model_simple, "simple.pth")
torch.save(model_simple_crop, "simple_crop.pth")
torch.save(model_simple_flip, "simple_flip.pth")
torch.save(model_simple_rotation, "simple_rotation.pth")
torch.save(model_simple_colorJitter, "simple_colorJitter.pth")
torch.save(model_simple_augmentation, "simple_augmentation.pth")

## Ensemble

In [24]:
def hard_ensemble(models, dataloader):
    all_preds = []
    all_labels = []

    device = next(models[0].parameters()).device

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Processing batches", unit="batch"):
            inputs = inputs.to(device)
            targets = targets.to(device)

            preds = torch.stack([torch.argmax(model(inputs), dim=1) for model in models], dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(targets.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    majority_preds = mode(all_preds, axis=1, keepdims=False).mode.squeeze()

    all_labels = np.concatenate(all_labels)

    accuracy = np.mean(majority_preds == all_labels)
    print(f"Majority Voting Ensemble Accuracy: {accuracy * 100:.2f}%")

    return majority_preds, all_labels


In [ ]:
def soft_ensemble(models, dataloader):
    all_preds = []
    all_labels = []

    device = next(models[0].parameters()).device

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Processing batches", unit="batch"):
            inputs = inputs.to(device)
            targets = targets.to(device)

            probs = [torch.softmax(model(inputs), dim=1) for model in models]
            all_preds.append(torch.stack(probs, dim=1))
            all_labels.append(targets)

    all_preds = torch.cat(all_preds, dim=0)
    avg_probs = torch.mean(all_preds, dim=1)
    soft_preds = torch.argmax(avg_probs, dim=1)
    all_labels = torch.cat(all_labels, dim=0)

    accuracy = torch.mean((soft_preds == all_labels).float())
    print(f"Soft Voting Ensemble Accuracy: {accuracy.item() * 100:.2f}%")

    return soft_preds.cpu().numpy(), all_labels.cpu().numpy()

In [16]:
model_paths = ["vgg_augmentation_advanced_trained.pth", "vgg_augmentation.pth", "vgg_improved_aug2.pth"]
models = [torch.load(path, weights_only=False, map_location=torch.device("cuda")) for path in model_paths]

for model in models:
    model.eval()

In [25]:
hard_ensemble(models, cinic_valid)

Processing batches: 100%|██████████| 704/704 [03:32<00:00,  3.32batch/s]


Majority Voting Ensemble Accuracy: 80.41%


(array([0, 0, 8, ..., 9, 9, 9], shape=(90000,)),
 array([0, 0, 0, ..., 9, 9, 9], shape=(90000,)))

In [17]:
soft_ensemble(models, cinic_valid)

Processing batches: 100%|██████████| 704/704 [02:59<00:00,  3.92batch/s]

Soft Voting Ensemble Accuracy: 81.40%


(array([0, 0, 8, ..., 9, 9, 9], shape=(90000,)),
 array([0, 0, 0, ..., 9, 9, 9], shape=(90000,)))